# German_Traffic_Sign_Recognition
## About Dataset
### Context
The [German Traffic Sign Benchmark](https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign?datasetId=82373)
 is a multi-class, single-image classification challenge held at the International Joint Conference on Neural Networks (IJCNN) 2011. We cordially invite researchers from relevant fields to participate: The competition is designed to allow for participation without special domain knowledge. Our benchmark has the following properties:

* Single-image, multi-class classification problem
* More than 40 classes
* More than 50,000 images in total
* Large, lifelike database

**Acknowledgements** \
INI Benchmark Website


## Imports

In [1]:
import logging
import os

# ignore all the warning and debug information from tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.getLogger('tensorflow').setLevel(logging.ERROR)

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from libs.nn.conv import CNN
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import SGD
from libs.preprocessing import ImageToArrayPreprocessor
from libs.preprocessing import SimplePreprocessor
from libs.datasets import SimpleDatasetLoader
import matplotlib.pyplot as plt
from libs.utils.myplot import plot_confusion_matrix
import matplotlib.image as img
import numpy as np
from imutils import paths
import os

## Prepare Dataset for training

In [2]:
# Look into the data directory
images_dir = './gtsrb-german-traffic-sign'


dataset_dir = f'{images_dir}/Train'
dataset_imgs = list(paths.list_images(dataset_dir))

In [3]:
BATCH_SIZE = 32
IMG_WIDTH = IMG_HEIGHT = 32
RANDOM_STATE=42

In [4]:
classes = {0: 'Speed limit (20km/h)',
           1: 'Speed limit (30km/h)',
           2: 'Speed limit (50km/h)',
           3: 'Speed limit (60km/h)',
           4: 'Speed limit (70km/h)',
           5: 'Speed limit (80km/h)',
           6: 'End of speed limit (80km/h)',
           7: 'Speed limit (100km/h)',
           8: 'Speed limit (120km/h)',
           9: 'No passing',
           10: 'No passing veh over 3.5 tons',
           11: 'Right-of-way at intersection',
           12: 'Priority road',
           13: 'Yield',
           14: 'Stop',
           15: 'No vehicles',
           16: 'Veh > 3.5 tons prohibited',
           17: 'No entry',
           18: 'General caution',
           19: 'Dangerous curve left',
           20: 'Dangerous curve right',
           21: 'Double curve',
           22: 'Bumpy road',
           23: 'Slippery road',
           24: 'Road narrows on the right',
           25: 'Road work',
           26: 'Traffic signals',
           27: 'Pedestrians',
           28: 'Children crossing',
           29: 'Bicycles crossing',
           30: 'Beware of ice/snow',
           31: 'Wild animals crossing',
           32: 'End speed + passing limits',
           33: 'Turn right ahead',
           34: 'Turn left ahead',
           35: 'Ahead only',
           36: 'Go straight or right',
           37: 'Go straight or left',
           38: 'Keep right',
           39: 'Keep left',
           40: 'Roundabout mandatory',
           41: 'End of no passing',
           42: 'End no passing veh > 3.5 tons'}

In [5]:
# initialize the image preprocessor
sp = SimplePreprocessor(IMG_WIDTH, IMG_HEIGHT)
iap = ImageToArrayPreprocessor()

# load the dataset from disk 
sdl = SimpleDatasetLoader(preprocessors=[sp, iap])

X, y = sdl.load(dataset_imgs, verbose=500)
X.shape, y.shape

[INFO] processed 500/39209
[INFO] processed 1000/39209
[INFO] processed 1500/39209
[INFO] processed 2000/39209
[INFO] processed 2500/39209
[INFO] processed 3000/39209
[INFO] processed 3500/39209
[INFO] processed 4000/39209
[INFO] processed 4500/39209
[INFO] processed 5000/39209
[INFO] processed 5500/39209
[INFO] processed 6000/39209
[INFO] processed 6500/39209
[INFO] processed 7000/39209
[INFO] processed 7500/39209
[INFO] processed 8000/39209
[INFO] processed 8500/39209
[INFO] processed 9000/39209
[INFO] processed 9500/39209
[INFO] processed 10000/39209
[INFO] processed 10500/39209
[INFO] processed 11000/39209
[INFO] processed 11500/39209
[INFO] processed 12000/39209
[INFO] processed 12500/39209
[INFO] processed 13000/39209
[INFO] processed 13500/39209
[INFO] processed 14000/39209
[INFO] processed 14500/39209
[INFO] processed 15000/39209
[INFO] processed 15500/39209
[INFO] processed 16000/39209
[INFO] processed 16500/39209
[INFO] processed 17000/39209
[INFO] processed 17500/39209
[INFO

((39209, 32, 32, 3), (39209,))

Normalization

In [6]:
X= X.astype("float") / 255.0
np.min(X), np.max(X)

(0.0, 1.0)

Split the dataset into train, and validation

In [7]:
# Separate 20% validation dataset
X_train, _X_valid, y_train, _y_valid = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    shuffle=True, 
                                                    stratify=y,
                                                    random_state=RANDOM_STATE 
                                                    )

X_train.shape, _X_valid.shape                   

((31367, 32, 32, 3), (7842, 32, 32, 3))

In [8]:
# Separate 30% dataset for testing from validation
X_valid, X_test, y_valid, y_test = train_test_split(_X_valid, _y_valid, 
                                                    test_size=0.30, 
                                                    random_state=RANDOM_STATE 
                                                    )
X_valid.shape, X_test.shape 

((5489, 32, 32, 3), (2353, 32, 32, 3))

In [9]:
len(np.unique(y))

43

In [10]:
y

array([ 6,  6,  6, ..., 29, 29, 29])

## Prepare CNN Model

In [11]:
cnn_model = CNN(width=IMG_WIDTH, heigth=IMG_HEIGHT, depth= 3, classes=len(np.unique(y)))

TypeError: CNN() takes no arguments

## Train the Model

## Evaluate the Model